# Laboratory work #1
For beginning importing libraries.

In [ ]:
# Import scikit-learn libraries
# Import preprocessing lib
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
# Import help function
from sklearn.utils.multiclass import type_of_target
from  sklearn.model_selection import train_test_split, GridSearchCV
# Import metrics
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
# Import models
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


# Import other libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

#### Downloading Dataset
Our dataset is Possible Asteroid Impacts with Earth (from [kaggle](https://www.kaggle.com/datasets/nasa/asteroid-impacts))

## Preprocessing Dataset

In [ ]:
# Loading dataset to pandas DataFrame
asteroid_df = pd.read_csv("datasets\\asteroid_dataset\\asteroid_classification.csv")

In [ ]:
# Getting data from dataframe
print(asteroid_df.head())
# Getting names of column
print("==>Column names<==")
print(f"=>asteroid columns: {asteroid_df.columns.tolist()}")
# Getting size of dataset
print("==>Dataset size<==")
print(f"asteroid shape={asteroid_df.shape}")

In [ ]:
# Getting info from dataframe
print(asteroid_df.info())

In [ ]:
# Drop unnecessary columns for analysis from the dataframe
asteroid_df.drop(["Object Name", "Epoch (TDB)", "Perihelion Argument (deg)", "Node Longitude (deg)",
                          "Mean Anomoly (deg)", "Perihelion Distance (AU)", "Aphelion Distance (AU)",
                          "Minimum Orbit Intersection Distance (AU)", "Orbital Reference"], axis=1, inplace=True)
print(asteroid_df.columns)

In [ ]:
# Drop null values of dataframe as we have only one null value
asteroid_df.dropna(inplace=True)
asteroid_df.info()

In [ ]:
# Drop the 90% of data in dataframe
asteroid_df_size = asteroid_df.size
asteroid_df = asteroid_df[:int(asteroid_df_size*0.1)]

In [ ]:
# Get the description of the dataframe
print(asteroid_df.describe())

In [ ]:
# View class names
print(set(asteroid_df["Object Classification"]))

In [ ]:
# View count of class names
print(asteroid_df.iloc[:, 0].value_counts())

In [ ]:
# Change class names
asteroid_df["Object Classification"].mask(asteroid_df["Object Classification"] == "Apollo Asteroid" ,
                                          "Apollo", inplace=True)
asteroid_df["Object Classification"].mask(asteroid_df["Object Classification"] == "Aten Asteroid",
                                          "Aten", inplace=True)
asteroid_df["Object Classification"].mask(asteroid_df["Object Classification"] == "Amor Asteroid",
                                          "Amor", inplace=True)
asteroid_df["Object Classification"].mask(asteroid_df["Object Classification"] == "Apohele Asteroid",
                                          "Apohele", inplace=True)
# Drop unnecessary class
necessary_class = ["Apollo", "Aten", "Amor"]
asteroid_df = asteroid_df[asteroid_df["Object Classification"].isin(necessary_class)]

In [ ]:
# View class names
print(set(asteroid_df["Object Classification"]))

In [ ]:
# View count of class names
print(asteroid_df.iloc[:, 0].value_counts())

In [ ]:
print(asteroid_df["Object Classification"].head())
print(asteroid_df.info())

## Creating functions for visualise data

In [ ]:
def boxplot_data(dataframe, features, target_features):
    """Function to show boxplot of dataframe;
    :param dataframe: pandas DataFrame with data;
    :param features: list with features name from dataframe;
    :param target_features: number of target feature.
    """
    # Define plot variables
    n = len(features)
    cols = 3
    rows = int(n/cols)
    features_iter = 0

    # Build plot
    fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(25, 23))
    for i in range(rows):
        for j in range(cols):
            if features_iter == target_features:
                features_iter += 1
                continue
            sns.boxplot(x=features[target_features], y=features[features_iter], data=dataframe, ax=ax[i][j])
            features_iter += 1

    plt.show()

def remove_outliers(dataframe, features):
    """Function to remove the outliers;
    :param dataframe: pandas DataFrame with data;
    :param features: list with features from dataframe.
    """
    # Copy dataframe to another variable
    dataframe_copy = dataframe.copy()

    # Iterate through features
    for feature in features:
        if dataframe[feature].dtype == object:
            continue
        # Calculate q1, q3 and iqr
        q3 = dataframe[feature].quantile(0.75)
        q1 = dataframe[feature].quantile(0.25)
        iqr = q3 - q1

        # Get local minimum and maximum
        local_min = q1 - (1.5 * iqr)
        local_max = q3 + (1.5 * iqr)

        # Remove the outliers
        dataframe_copy = dataframe_copy[(dataframe_copy[feature] >= local_min) &
                                        (dataframe_copy[feature] <= local_max)]

    return dataframe_copy

def histogram_data(dataframe, features):
    """Function to show histogram of dataframe;
    :param dataframe: pandas DataFrame with data;
    :param features: list with features from dataframe.
    """
    # Define plot variables
    n = len(features)
    cols = 3
    rows = int(n/cols)
    features_iter = 1

    # Build plot
    fig, ax = plt.subplots(ncols=cols, figsize=(24, 20))
    for i in range(rows):
        for j in range(cols):
            if len(features) <= features_iter:
                break
            sns.histplot(x=features[features_iter], y=features[features_iter+1],
                         data=dataframe, ax=ax[j])
            features_iter += 1
            plt.show()


## Visualising Dataset

In [ ]:
# Visualising with heatmap
plt.figure(figsize=(17, 12))
heat_map = sns.heatmap(asteroid_df.corr(numeric_only=True), cmap="coolwarm", linewidths=1, annot=True, fmt=".2f")
plt.title("HeatMap")
plt.show()

In [ ]:
# Extract features from the df
asteroid_features = asteroid_df.columns.tolist()
print(asteroid_features)

In [ ]:
# Show boxplot to check for outliers
boxplot_data(asteroid_df, asteroid_features, 0)

In [ ]:
# Remove outliers from the dataframe
asteroid_df = remove_outliers(asteroid_df, asteroid_features)

# Show the boxplot
boxplot_data(asteroid_df, asteroid_features, 0)

In [ ]:
# Show histogram plot for our dataframe
histogram_data(asteroid_df, asteroid_features)

In [ ]:
# Show pair plot for our features
sns.pairplot(asteroid_df, hue=asteroid_features[0], palette="Set1")

## Preprocess Dataset for model

In [57]:
# View class names
class_names = list(set(asteroid_df["Object Classification"]))
print(class_names)

['Aten', 'Amor', 'Apollo']


### Normalizing Dataset

In [ ]:
# Create normalized dataset
norm_asteroid_df = asteroid_df.copy()
# apply normalization techniques
for column in norm_asteroid_df:
    if norm_asteroid_df[column].dtype == object:
        continue
    norm_asteroid_df[column] = norm_asteroid_df[column] / norm_asteroid_df[column].abs().max()

In [ ]:
# View normalized Dataset
print(norm_asteroid_df.head())

In [ ]:
# View info about Dataset
norm_asteroid_df.info()

### Numerizing Dataset

In [ ]:
# Edit name of column "Object Classification" with _
# For using this name in next cell
number_asteroid_df = norm_asteroid_df.copy()
number_asteroid_df.rename(columns={"Object Classification": "Object_Classification"}, inplace=True)

In [ ]:
# Replace string class to numbers
obj_class = {"Apollo": 0, "Aten": 1, "Amor": 2}
number_asteroid_df.Object_Classification = [obj_class[item] for item in number_asteroid_df.Object_Classification]

In [ ]:
# View new dataset
print(number_asteroid_df.head())

### One-Hot Encoding Dataset

In [ ]:
# One-hot Encoding the Object Classification Feature
one_hot = OneHotEncoder()
# Copy our dataset
onehot_asteroid_df = norm_asteroid_df.copy()
# Fitting one-hot encoder
encoded = one_hot.fit_transform(onehot_asteroid_df[["Object Classification"]])
onehot_asteroid_df[one_hot.categories_[0]] = encoded.toarray()
# Drop unnecessary "Object Classification" feature
onehot_asteroid_df.drop(["Object Classification"], axis=1, inplace=True)
print(onehot_asteroid_df.head())

In [ ]:
# Change data type in one-hot encoded column
column_dtype_dict = {"Amor": int,
                     "Apollo": int,
                     "Aten": int}
norm_onehot_asteroid_df = onehot_asteroid_df.astype(column_dtype_dict)
print(norm_onehot_asteroid_df.dtypes)
print(norm_onehot_asteroid_df.head())

### Split Dataset to Train & Test

In [ ]:
# Split Categorical Dataset
x = norm_asteroid_df.drop(["Object_Classification"], axis=1)
y = norm_asteroid_df["Object_Classification"]
# Split to train test sets
catg_X_train, catg_X_test, catg_y_train, catg_y_test = train_test_split(x, y, test_size=0.20)

In [ ]:
# View x_train & y_train
print(f"X-train:\n{catg_X_train.head()}")
print(f"Y-train:\n{catg_y_train.head()}")

In [ ]:
# Split Numeric Dataset
x = number_asteroid_df.drop(["Object_Classification"], axis=1)
y = number_asteroid_df["Object_Classification"]
# Split to train test sets
num_X_train, num_X_test, num_y_train, num_y_test = train_test_split(x, y, test_size=0.20)

In [ ]:
# View x_train & y_train
print(f"X-train:\n{num_X_train.head()}")
print(f"Y-train:\n{num_y_train.head()}")

In [ ]:
# Split One-Hot Dataset
x = norm_onehot_asteroid_df.drop(["Apollo", "Aten", "Amor"], axis=1)
y = norm_onehot_asteroid_df[["Apollo", "Aten", "Amor"]]
# Split to train test sets
oneh_X_train, oneh_X_test, oneh_y_train, oneh_y_test = train_test_split(x, y, test_size=0.20)

In [ ]:
# View x_train & y_train
print(f"X-train:\n{oneh_X_train.head()}")
print(f"Y-train:\n{oneh_y_train.head()}")

In [ ]:
# View splitting result
print(type(catg_X_train), type(catg_y_train))
print(catg_X_train.shape, catg_y_train.shape)

## Creating Help functions for evaluating model

In [ ]:
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False):
    """Makes a labelled confusion matrix comparing predictions and ground truth labels.

    If classes is passed, confusion matrix will be labelled, if not, integer class values
    will be used;
    :param y_true: Array of truth labels (must be same shape as y_pred);
    :param y_pred: Array of predicted labels (must be same shape as y_true);
    :param classes: Array of class labels (e.g. string form). If `None`, integer labels are used;
    :param figsize: Size of output figure (default=(10, 10));
    :param text_size: Size of output figure text (default=15);
    :param norm: normalize values or not (default=False);
    :param savefig: save confusion matrix to file (default=False).

    :returns: A labelled confusion matrix plot comparing y_true and y_pred.

    Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
    """
    # Create the confusion matrix
    type_true = type_of_target(y_true)
    if type_true == "multilabel-indicator":
        confusion_mx = multilabel_confusion_matrix(y_true, y_pred)
        print(confusion_mx)
    else:
        confusion_mx = confusion_matrix(y_true, y_pred)
    confusion_mx_norm = confusion_mx.astype("float") / confusion_mx.sum(axis=1)[:, np.newaxis] # normalize it
    n_classes = confusion_mx.shape[0] # find the number of classes we're dealing with

    # Plot the figure and make it pretty
    fig, ax = plt.subplots(figsize=figsize)
    cax = ax.matshow(confusion_mx, cmap=plt.cm.Greens) # colors will represent how 'correct' a class is, darker == better
    fig.colorbar(cax)

    # Are there a list of classes?
    if classes:
        labels = classes
    else:
        labels = np.arange(confusion_mx.shape[0])

    # Label the axes
    ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes),
         xticklabels=labels, # axes will label with class names (if they exist) or ints
         yticklabels=labels)

    # Make x-axis labels appear on bottom
    ax.xaxis.set_label_position("bottom")
    ax.xaxis.tick_bottom()

    # Set the threshold for different colors
    threshold = (confusion_mx.max() + confusion_mx.min()) / 2.

    # Plot the text on each cell
    for i, j in itertools.product(range(confusion_mx.shape[0]), range(confusion_mx.shape[1])):
        if norm:
            plt.text(j, i, f"{confusion_mx[i, j]} ({confusion_mx_norm[i, j]*100:.1f}%)",
                     horizontalalignment="center",
                     color="white" if confusion_mx[i, j] > threshold else "black",
                     size=text_size)
        else:
            plt.text(j, i, f"{confusion_mx[i, j]}",
                     horizontalalignment="center",
                     color="white" if confusion_mx[i, j] > threshold else "black",
                     size=text_size)

    # Save the figure to the current working directory
    if savefig:
        fig.savefig("confusion_matrix.png")


def classification_confusion_evaluation(y_true, y_pred, classes=None):
    """

    :param y_true:
    :param y_pred:
    :param classes:
    """
    model_accuracy = accuracy_score(y_true, y_pred)

    print("==>Classification Report<==")
    print(classification_report(y_true, y_pred))
    print("==>Confusion Matrix<==")
    make_confusion_matrix(y_true, y_pred, classes)
    print("==========================")
    print("==>Model accuracy: {:.2f}".format(model_accuracy))

## Training Classification Algorithms

### Decision Tree

In [ ]:
# Define Decision Tree with "gini" criteria and max depth of tree as 7
categorical_tree = DecisionTreeClassifier(criterion="gini", max_depth=7)
# Train the model on categorical data
categorical_tree.fit(catg_X_train, catg_y_train)

In [ ]:
# Define Decision Tree with "gini" criteria and max depth of tree as 7
numeric_tree = DecisionTreeClassifier(criterion="gini", max_depth=7)
# Train the model on categorical data
numeric_tree.fit(num_X_train, num_y_train)

In [ ]:
# Define Decision Tree with "gini" criteria and max depth of tree as 7
one_hot_tree = DecisionTreeClassifier(criterion="gini", max_depth=7)
# Train the model on categorical data
one_hot_tree.fit(oneh_X_train, oneh_y_train)

#### Taking predictions

In [ ]:
# Take a prediction from the categorical data
catg_tree_pred = categorical_tree.predict(catg_X_test)

In [ ]:
# Take a prediction from the numeric data
num_tree_pred = numeric_tree.predict(num_X_test)

In [ ]:
# Take a prediction from the one-hot data
oneh_tree_pred = one_hot_tree.predict(oneh_X_test)

#### Tree Visualization

In [ ]:
# Make visualization of our Decision Tree model
plt.figure(figsize=(80, 80))
plot_tree(categorical_tree, feature_names=["Orbit Axis (AU)", "Orbit Eccentricity", "Orbit Inclination (deg)",
                                "Orbit Period (yr)", "Asteroid Magnitude"],
          class_names=["Apollo", "Aten", "Amor"])
plt.show()

### Random Forest

In [ ]:
# Define Random Forest model (with 100 random tree and random state 17)
categorical_forest = RandomForestClassifier(n_estimators=100,
                                            random_state=17)
# Train the model
categorical_forest.fit(catg_X_train, catg_y_train)

In [ ]:
# Define Random Forest model (with 100 random tree and random state 17)
numeric_forest = RandomForestClassifier(n_estimators=100,
                                        random_state=17)
# Train the model
numeric_forest.fit(num_X_train, num_y_train)

In [ ]:
# Define Random Forest model (with 100 random tree and random state 17)
one_hot_forest = RandomForestClassifier(n_estimators=100,
                                        random_state=17)
# Train the model
one_hot_forest.fit(oneh_X_train, oneh_y_train)

#### Taking predictions

In [ ]:
# Take a prediction from the model
catg_forest_pred = categorical_forest.predict(catg_X_test)

In [ ]:
# Take a prediction from the model
num_forest_pred = numeric_forest.predict(num_X_test)

In [ ]:
# Take a prediction from the model
oneh_forest_pred = one_hot_forest.predict(oneh_X_test)

### AdaBoost

In [ ]:
# Define AdaBoost model
catg_adab = AdaBoostClassifier(n_estimators=100,
                              random_state=17)
# Train the model
catg_adab.fit(catg_X_train, catg_y_train)

In [ ]:
# Define AdaBoost model
num_adab = AdaBoostClassifier(n_estimators=100,
                              random_state=17)
# Train the model
num_adab.fit(num_X_train, num_y_train)

In [ ]:
# Define AdaBoost model
oneh_adab = AdaBoostClassifier(n_estimators=100,
                              random_state=17)
# Train the model
oneh_adab.fit(oneh_X_train, oneh_y_train)

#### Tacking predictions

In [ ]:
# Take a prediction from the model
catg_adab_pred = catg_adab.predict(catg_X_test)

In [ ]:
# Take a prediction from the model
num_adab_pred = num_adab.predict(num_X_test)

In [ ]:
# Take a prediction from the model
oneh_adab_pred = oneh_adab.predict(oneh_X_test)

### kNN (K Nearest Neighbors)

#### Define Scaler

In [ ]:
# Standardize the variables
# Define scaler
catg_scaler = StandardScaler()
# Fit the scaler
catg_scaler.fit(catg_X_train)
# Maker scaled data
scaled_catg_X_train = catg_scaler.transform(catg_X_train)
scaled_catg_X_test = catg_scaler.transform(catg_X_test)

In [ ]:
# Standardize the variables
# Define scaler
num_scaler = StandardScaler()
# Fit the scaler
num_scaler.fit(num_X_train)
# Maker scaled data
scaled_num_X_train = num_scaler.transform(num_X_train)
scaled_num_X_test = num_scaler.transform(num_X_test)

In [ ]:
# Standardize the variables
# Define scaler
oneh_scaler = StandardScaler()
# Fit the scaler
oneh_scaler.fit(oneh_X_train)
# Maker scaled data
scaled_oneh_X_train = oneh_scaler.transform(oneh_X_train)
scaled_oneh_X_test = oneh_scaler.transform(oneh_X_test)

#### Choosing optimal parameter

In [ ]:
# Make array with error rate (Categorical)
error_rate = []
for i in range(1, 30):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(scaled_catg_X_train, catg_y_train)
    pred_i = knn.predict(scaled_catg_X_test)
    error_rate.append(np.mean(pred_i != catg_y_test))
# Plotting results of error for all neighbors
plt.figure(figsize=(10, 6))
plt.plot(range(1, 30), error_rate, marker="o", markersize=10)
plt.title("Error Rate vs. K Value")
plt.xlabel("K")
plt.ylabel("Error Rate")
plt.grid()

In [ ]:
# Make array with error rate (Numerical)
error_rate = []
for i in range(1, 30):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(scaled_num_X_train, num_y_train)
    pred_i = knn.predict(scaled_num_X_test)
    error_rate.append(np.mean(pred_i != num_y_test))
# Plotting results of error for all neighbors
plt.figure(figsize=(10, 6))
plt.plot(range(1, 30), error_rate, marker="o", markersize=10)
plt.title("Error Rate vs. K Value")
plt.xlabel("K")
plt.ylabel("Error Rate")
plt.grid()

In [ ]:
# Make array with error rate (One-Hot)
error_rate = []
for i in range(1, 30):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(scaled_oneh_X_train, oneh_y_train)
    pred_i = knn.predict(scaled_oneh_X_test)
    error_rate.append(np.mean(pred_i != oneh_y_test))
# Plotting results of error for all neighbors
plt.figure(figsize=(10, 6))
plt.plot(range(1, 30), error_rate, marker="o", markersize=10)
plt.title("Error Rate vs. K Value")
plt.xlabel("K")
plt.ylabel("Error Rate")
plt.grid()

#### Define and train model

In [ ]:
# Define kNN model
catg_knn = KNeighborsClassifier(n_neighbors=13)
# Train the model
catg_knn.fit(scaled_catg_X_train, catg_y_train)

In [ ]:
# Define kNN model
num_knn = KNeighborsClassifier(n_neighbors=13)
# Train the model
num_knn.fit(scaled_num_X_train, num_y_train)

In [ ]:
# Define kNN model with new optimal parameter n_neighbors for best result
oneh_knn = KNeighborsClassifier(n_neighbors=13)
# Train the model
oneh_knn.fit(scaled_oneh_X_train, oneh_y_train)

#### Taking predictions

In [ ]:
# Take a prediction from the model
catg_knn_pred = catg_knn.predict(scaled_catg_X_test)

In [ ]:
# Take a prediction from the model
num_knn_pred = num_knn.predict(scaled_num_X_test)

In [ ]:
oneh_knn_pred = oneh_knn.predict(scaled_oneh_X_test)

### SVM (Support Vector Machine)

In [ ]:
# Define SVM model
catg_svm = SVC(C=1.0, gamma=1.0, kernel='rbf')
# Train the model
catg_svm.fit(scaled_catg_X_train, catg_y_train)

In [ ]:
# Define SVM model
svm_model = SVC(C=1.0, gamma=1.0, kernel='rbf')
# Train the model
svm_model.fit(scaled_num_X_train, num_y_train)

In [ ]:
# Define SVM model
svm_model = SVC(C=1.0, gamma=1.0, kernel='rbf')
# Train the model
svm_model.fit(scaled_oneh_X_train, oneh_y_train)

#### Tacking predictions

In [ ]:
# Take a prediction from the model
svm_y_pred = svm_model.predict(catg_X_test)

In [ ]:
# Take a prediction from the model
svm_y_pred = svm_model.predict(num_X_test)

In [ ]:
# Take a prediction from the model
svm_y_pred = svm_model.predict(oneh_X_test)

### Choosing optimal parameter

In [ ]:
# Make Gridsearch for find optimal parameters for SVM
param_grid = {"C": [0.1, 1, 10, 100, 1000],
              "gamma": [1, .1, .01, .001, .0001],
              "kernel": ["rbf"]}

In [ ]:
# Define GridSearch
grid_search = GridSearchCV(SVC(), param_grid, refit=True)

In [ ]:
# Fit the grid
grid_search.fit(scaled_X_train, y_train)

In [ ]:
# View optimal parameters for SVM
grid_search.best_params_

In [ ]:
# View best estimation parameters
grid_search.best_estimator_

In [ ]:
# Take a grid predictions
grid_pred = grid_search.predict(scaled_X_test)

## Choosing best model (Evaluating)
View model classification report (with precision, recall and f1-score), confusion matrix and accuracy score.

### Decision Tree

In [ ]:
# Categorical Data
classification_confusion_evaluation(y_test, forest_mult_y_pred, class_names)

In [ ]:
# Number Data
classification_confusion_evaluation(y_test, forest_mult_y_pred, class_names)

In [ ]:
# One-Hot Encoded Data
classification_confusion_evaluation(y_test, forest_mult_y_pred, class_names)

### Random Forest

In [ ]:
# Categorical Data
classification_confusion_evaluation(y_test, forest_mult_y_pred, class_names)

In [ ]:
# Number Data
classification_confusion_evaluation(y_test, forest_mult_y_pred, class_names)

In [ ]:
# One-Hot Encoded Data
classification_confusion_evaluation(y_test, forest_mult_y_pred, class_names)

### AdaBoost

In [ ]:
# Categorical Data
classification_confusion_evaluation(y_test, forest_mult_y_pred, class_names)

In [ ]:
# Number Data
classification_confusion_evaluation(y_test, forest_mult_y_pred, class_names)

In [ ]:
# One-Hot Encoded Data
classification_confusion_evaluation(y_test, forest_mult_y_pred, class_names)

### kNN

In [ ]:
# Categorical Data
classification_confusion_evaluation(y_test, forest_mult_y_pred, class_names)

In [ ]:
# Number Data
classification_confusion_evaluation(y_test, forest_mult_y_pred, class_names)

In [ ]:
# One-Hot Encoded Data
classification_confusion_evaluation(y_test, forest_mult_y_pred, class_names)

### SVM

In [ ]:
# Categorical Data
classification_confusion_evaluation(y_test, forest_mult_y_pred, class_names)

In [ ]:
# Number Data
classification_confusion_evaluation(y_test, forest_mult_y_pred, class_names)

In [ ]:
# One-Hot Encoded Data
classification_confusion_evaluation(y_test, forest_mult_y_pred, class_names)

### GridSearch (SVM)

In [ ]:
# Categorical Data
classification_confusion_evaluation(y_test, forest_mult_y_pred, class_names)

In [ ]:
# Number Data
classification_confusion_evaluation(y_test, forest_mult_y_pred, class_names)

In [ ]:
# One-Hot Encoded Data
classification_confusion_evaluation(y_test, forest_mult_y_pred, class_names)

### Best model is